In [ ]:
# === 라벨만(.txt) 있는 파일 찾기 (이미지 매칭 없음) ==========================
from pathlib import Path

ROOT = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20250909_merge_data")

# 이미지/라벨 확장자 정의 (필요시 추가)
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp", ".tif", ".tiff", ".gif"}
LABEL_EXT = ".txt"

# 전체 스캔
all_files = [p for p in ROOT.rglob("*") if p.is_file()]
images = [p for p in all_files if p.suffix.lower() in IMG_EXTS]
labels = [p for p in all_files if p.suffix.lower() == LABEL_EXT]

# 전역(stem 기준) 이미지 존재 여부 판단
stems_img = {p.stem for p in images}

label_only = [lbl for lbl in labels if lbl.stem not in stems_img]

# 리포트
print("====================================")
print("[검증 결과]")
print(f"- 경로: {ROOT}")
print(f"- 이미지 수: {len(images)}")
print(f"- 라벨(.txt) 수: {len(labels)}")
print(f"- '라벨만 있는' 파일 수: {len(label_only)}")
print("====================================")

# 목록 출력 (상대경로로 보기 좋게)
for p in label_only:
    try:
        rel = p.relative_to(ROOT)
    except Exception:
        rel = p
    print(" -", rel)

# # 파일로 저장
# out_path = ROOT / "label_only_list.txt"
# out_path.write_text("\n".join(str(p.relative_to(ROOT)) for p in label_only), encoding="utf-8")
print(f"\n[SAVED] 라벨만 있는 파일 목록: {out_path}")

In [ ]:
# === 두 경로의 YOLO 데이터 머지(복사) + 매칭/충돌 리포트 ==========================
from pathlib import Path
import shutil
from collections import defaultdict

# ===== 설정 =====
SRC1 = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250700/good_data_fix_label_processing")
SRC2 = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_good_data")
DST  = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20250909_merge_data")

IMG_EXTS_PRIORITY = [".jpg", ".jpeg", ".png", ".bmp", ".webp", ".tif", ".tiff"]
IMG_EXTS = set(IMG_EXTS_PRIORITY)
LABEL_EXT = ".txt"

# ===== 유틸 =====
def is_image(p: Path) -> bool:
    return p.suffix.lower() in IMG_EXTS

def is_label(p: Path) -> bool:
    return p.suffix.lower() == LABEL_EXT

def best_image_choice(paths):
    """동일 stem의 다중 이미지가 있다면 우선순위 확장자 기준으로 1개 선택"""
    if len(paths) == 1:
        return paths[0]
    def keyfunc(p: Path):
        try:
            return IMG_EXTS_PRIORITY.index(p.suffix.lower())
        except ValueError:
            return len(IMG_EXTS_PRIORITY)
    return sorted(paths, key=keyfunc)[0]

def collect_items(src_dir: Path):
    """src_dir 재귀 탐색 → {stem: img_path or None, label_path or None} 목록"""
    images_by_stem = defaultdict(list)
    labels_by_stem = {}
    for p in src_dir.rglob("*"):
        if not p.is_file():
            continue
        if is_image(p):
            images_by_stem[p.stem].append(p)
        elif is_label(p):
            # 동일 stem 다수 라벨이 있으면 첫 번째만 사용
            labels_by_stem.setdefault(p.stem, p)

    items = []
    all_stems = set(images_by_stem.keys()) | set(labels_by_stem.keys())
    for s in sorted(all_stems):
        img_paths = images_by_stem.get(s, [])
        img = best_image_choice(img_paths) if img_paths else None
        lbl = labels_by_stem.get(s)
        items.append({"stem": s, "img": img, "lbl": lbl, "src": src_dir})
    return items

def next_available_stem(dst_dir: Path, base_stem: str, need_img_ext: str|None, need_lbl: bool):
    """DST 내에서 충돌 없도록 stem_1, stem_2 ... 생성"""
    cand = base_stem
    i = 1
    while True:
        img_ok = True
        lbl_ok = True
        if need_img_ext:
            img_ok = not (dst_dir / f"{cand}{need_img_ext}").exists()
        if need_lbl:
            lbl_ok = not (dst_dir / f"{cand}{LABEL_EXT}").exists()
        if img_ok and lbl_ok:
            return cand
        i += 1
        cand = f"{base_stem}_{i}"

def copy_safe(src: Path, dst: Path):
    dst.parent.mkdir(parents=True, exist_ok=True)
    shutil.copy2(str(src), str(dst))

def show_samples(title, arr, maxn=5):
    print(f"\n[{title}] (최대 {maxn}개 예시)")
    for s in arr[:maxn]:
        print(" -", s)

# ===== 수집 =====
items1 = collect_items(SRC1)
items2 = collect_items(SRC2)
items = items1 + items2

# ===== 복사 수행 =====
DST.mkdir(parents=True, exist_ok=True)

pairs, img_only, lbl_only = 0, 0, 0
copied_imgs, copied_lbls = 0, 0
renamed = []   # (old_stem -> new_stem, src)
name_map_samples = []

for it in items:
    stem = it["stem"]
    img  = it["img"]
    lbl  = it["lbl"]

    need_img = img is not None
    need_lbl = lbl is not None
    if need_img and need_lbl:
        pairs += 1
    elif need_img:
        img_only += 1
    elif need_lbl:
        lbl_only += 1

    img_ext = img.suffix.lower() if img else None

    # 충돌 방지용 stem 선택 (이미지/라벨 모두 고려)
    dest_stem = next_available_stem(DST, stem, img_ext, need_lbl)

    # 리네임 여부 기록
    if dest_stem != stem:
        renamed.append((stem, dest_stem, it["src"]))
        if len(name_map_samples) < 8:
            name_map_samples.append(f"{it['src'].name}: {stem}  →  {dest_stem}")

    # 실제 복사
    if need_img:
        dst_img = DST / f"{dest_stem}{img_ext}"
        copy_safe(img, dst_img)
        copied_imgs += 1
    if need_lbl:
        dst_lbl = DST / f"{dest_stem}{LABEL_EXT}"
        copy_safe(lbl, dst_lbl)
        copied_lbls += 1

# ===== 리포트 =====
print("====================================")
print("[MERGE RESULT]")
print(f"- SRC1: {SRC1}")
print(f"- SRC2: {SRC2}")
print(f"- DST : {DST}")
print("------------------------------------")
print(f"- 총 stem 개수           : {len(items)}")
print(f"- 짝(이미지+라벨)         : {pairs}")
print(f"- 이미지만 있음           : {img_only}")
print(f"- 라벨만 있음             : {lbl_only}")
print("------------------------------------")
print(f"- 복사된 이미지 수        : {copied_imgs}")
print(f"- 복사된 라벨(.txt) 수    : {copied_lbls}")
print(f"- 이름 충돌로 리네임된 건 : {len(renamed)}")
print("====================================")

show_samples("리네임 매핑 예시 (old → new)", name_map_samples)


In [ ]:
from pathlib import Path
from collections import Counter

# ====== 경로 설정 ======
LABEL_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20250909_merge_data")

# YOLO 클래스 이름 매핑
classes = [
    "Divot",        # 0
    "Fixed_Divot",  # 1
    "Dieased_Grass",# 2
    "Pole",         # 3
    "Obstacle",     # 4
    "Sprinkler",    # 5
    "Drain",        # 6
    "Golf ball"     # 7
]

# ====== 객체 카운트 ======
counter = Counter()

# txt 라벨 파일 순회
for txt_file in LABEL_DIR.glob("*.txt"):
    with open(txt_file, "r") as f:
        for line in f:
            parts = line.strip().split()
            if not parts:
                continue
            cls_id = int(parts[0])
            counter[cls_id] += 1

# ====== 결과 출력 ======
print("=== 클래스별 객체 개수 ===")
for i, cls_name in enumerate(classes):
    print(f"{cls_name:<15}: {counter.get(i, 0)}")

print("\n총 객체 수:", sum(counter.values()))

In [ ]:
# augment_balance_final_custom.py
# - 사용자가 원하는 TARGET(클래스별 최종 객체 수)을 직접 설정
# - 재사용 제한을 클래스별로 가산할 수 있도록 개선
# - 나머지 안전장치(yolo_sanitize, 빈/이상 라인 제거, 로그 등) 유지

from pathlib import Path
from collections import Counter, defaultdict
import random, csv
import cv2
import albumentations as A

# ========== 사용자 설정 ==========
BASE = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20250909_merge_data")
OUT_DIR = BASE / "aug_balanced"     # 단일 폴더 (이미지/라벨 저장)
IMG_EXTS = [".jpg", ".jpeg", ".png"]
SEED = 42
random.seed(SEED)

# (필수) 최종 목표 개수를 직접 지정하세요. (원본처럼!!)
# 예시: Divot 3500, Fixed_Divot 3500, Dieased_Grass 300, 나머지는 기존 유지 or 적절히 설정
# 클래스 매핑: 0 Divot, 1 Fixed_Divot, 2 Dieased_Grass, 3 Pole, 4 Obstacle, 5 Sprinkler, 6 Drain, 7 Golf ball
TARGET = {
    0: 4500, # Divot
    1: 4500, # Fixed_Divot
    2: 600,  # Dieased_Grass
    3: 150,   # Pole (예시)
    4: 0,    # Obstacle (예시)
    5: 40,   # Sprinkler (예시)
    6: 150,   # Drain (예시)
    7: 180,   # Golf ball (예시)
}
ALL_CLASS_IDS = list(range(8))
ALLOW_IDS = set(ALL_CLASS_IDS)

# 재사용 제한 정책
# - 기본 회수
MAX_USES_BASE = 2
# - 클래스별 가산치(해당 클래스를 포함하고 "아직 부족"한 경우에만 적용)
#   예: 2와 3이 특히 부족하면 2회 더 허용, 5는 1회 더 허용… 필요에 맞게 수정
MAX_USES_BOOST_PER_CLASS = {
    0: 3,
    1: 3,
    2: 5,   # Dieased_Grass 부족하면 +2
    3: 5,
    4: 5,
    5: 5,   # Pole 부족하면 +2
    6: 5,   # Sprinkler 부족하면 +1
    7: 5,
    # 필요시 0..7 아무거나 추가/수정
}

# 배경(라벨 없는 이미지) 증강 배수
# 값=2 → 원본 1 + 증강 2 = 총 3배
BG_AUG_MULTIPLIER = 2

# 증강 파이프라인(색/조명 중심, 약한 변형)
aug = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=(-0.12, 0.18),
                               contrast_limit=(-0.12, 0.18), p=0.9),
    A.HueSaturationValue(hue_shift_limit=2, sat_shift_limit=12, val_shift_limit=10, p=0.8),
    A.RGBShift(r_shift_limit=8, g_shift_limit=6, b_shift_limit=6, p=0.5),
    A.RandomGamma(gamma_limit=(90, 120), p=0.4),
    A.OneOf([
        A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=0.5),
        A.ISONoise(color_shift=(0.01, 0.05), intensity=(0.08, 0.25), p=0.5),
        A.NoOp()
    ], p=0.4),
], bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"], min_visibility=0.25))

# ========== 유틸 ==========
def ensure_out_dir():
    OUT_DIR.mkdir(parents=True, exist_ok=True)

def read_yolo_label(lbl: Path):
    boxes, cls = [], []
    if not lbl.exists():
        return boxes, cls
    with open(lbl, "r") as f:
        for line in f:
            p = line.strip().split()
            if len(p) < 5:
                continue
            try:
                c = int(float(p[0]))
            except Exception:
                continue
            try:
                x, y, w, h = map(float, p[1:5])
            except Exception:
                continue
            x = min(max(x, 0.0), 1.0)
            y = min(max(y, 0.0), 1.0)
            w = min(max(w, 0.0), 1.0)
            h = min(max(h, 0.0), 1.0)
            if w <= 0.0 or h <= 0.0:
                continue
            boxes.append([x, y, w, h]); cls.append(c)
    return boxes, cls

def write_yolo_label(lbl_path: Path, boxes, cls, *, allow_ids=ALLOW_IDS):
    lbl_path.parent.mkdir(parents=True, exist_ok=True)
    with open(lbl_path, "w") as f:
        for c, (x, y, w, h) in zip(cls, boxes):
            try:
                c_int = int(float(c))
            except Exception:
                continue
            if allow_ids is not None and c_int not in allow_ids:
                continue
            x = float(max(0.0, min(1.0, x)))
            y = float(max(0.0, min(1.0, y)))
            w = float(max(0.0, min(1.0, w)))
            h = float(max(0.0, min(1.0, h)))
            if w <= 0.0 or h <= 0.0:
                continue
            f.write(f"{c_int} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")

def yolo_sanitize(boxes):
    eps = 1e-6
    out = []
    for x, y, w, h in boxes:
        l = x - w/2.0; r = x + w/2.0
        t = y - h/2.0; b = y + h/2.0
        l = max(0.0, min(1.0, l))
        r = max(0.0, min(1.0, r))
        t = max(0.0, min(1.0, t))
        b = max(0.0, min(1.0, b))
        if r - l <= eps or b - t <= eps:
            continue
        nx = (l + r) / 2.0
        ny = (t + b) / 2.0
        nw = (r - l)
        nh = (b - t)
        nx = min(max(nx, eps), 1.0 - eps)
        ny = min(max(ny, eps), 1.0 - eps)
        nw = min(max(nw, eps), 1.0 - eps)
        nh = min(max(nh, eps), 1.0 - eps)
        out.append([nx, ny, nw, nh])
    return out

def find_image_for_label(lbl: Path):
    stem = lbl.stem
    for ext in IMG_EXTS:
        p = lbl.with_suffix(ext)
        if p.exists(): return p
    for parent in [lbl.parent, *lbl.parents]:
        img_dir = parent / "images"
        if img_dir.exists():
            for ext in IMG_EXTS:
                p = img_dir / f"{stem}{ext}"
                if p.exists(): return p
    for ext in IMG_EXTS:
        hits = list(BASE.rglob(f"{stem}{ext}"))
        if hits: return hits[0]
    return None

def current_counts(root: Path):
    cnt = Counter()
    for lbl in root.rglob("*.txt"):
        _, cls = read_yolo_label(lbl)
        cnt.update(cls)
    return cnt

def collect_background_images(base: Path):
    label_stems = {p.stem for p in base.rglob("*.txt")}
    bg_images = []
    for ext in IMG_EXTS:
        for img in base.rglob(f"*{ext}"):
            if img.stem not in label_stems:
                bg_images.append(img)
    return sorted(bg_images)

def load_image(path: Path):
    img = cv2.imread(str(path))
    if img is None:
        raise RuntimeError(f"Failed to read image: {path}")
    return img

def next_unique_name(stem: str, used: set, tag: str):
    i = 1
    while True:
        s = f"{stem}_{tag}{i:04d}"
        if s not in used:
            used.add(s)
            return s
        i += 1

def effective_cap_for_file(file_cls_counter: Counter, deficit: Counter):
    """
    이 파일(라벨)의 클래스 구성과 현재 부족분을 보고
    MAX_USES_BASE + (부족 클래스 중 최대 가산치) 를 반환.
    """
    cap = MAX_USES_BASE
    best_boost = 0
    for c, n in file_cls_counter.items():
        if n <= 0: 
            continue
        if deficit.get(c, 0) > 0:
            boost = MAX_USES_BOOST_PER_CLASS.get(c, 0)
            if boost > best_boost:
                best_boost = boost
    return cap + best_boost

# ========== 실행 ==========
ensure_out_dir()

# 1) 배경 3배(원본 + 증강2)
bg_list = collect_background_images(BASE)
print(f"Background images: {len(bg_list)}")

used_names = set()
for bg_img in bg_list:
    try:
        img = load_image(bg_img)
    except Exception:
        continue
    for _ in range(BG_AUG_MULTIPLIER):
        out_stem = next_unique_name(bg_img.stem, used_names, tag="bg")
        out_img = OUT_DIR / f"{out_stem}{bg_img.suffix.lower()}"
        out_lbl = OUT_DIR / f"{out_stem}.txt"
        transformed = aug(image=img, bboxes=[], class_labels=[])
        aug_img = transformed["image"]
        cv2.imwrite(str(out_img), aug_img)
        with open(out_lbl, "w") as f:
            pass
print("Background augmentation done.")

# 2) 라벨 쌍 후보
label_files = sorted(BASE.rglob("*.txt"))
candidates = []  # (img_path, lbl_path, file_counts, stem)
skipped = 0
for lbl in label_files:
    boxes, cls_list = read_yolo_label(lbl)
    if not cls_list:
        continue
    img = find_image_for_label(lbl)
    if img is None or not img.exists():
        skipped += 1
        continue
    fcnt = Counter(cls_list)
    candidates.append((img, lbl, fcnt, lbl.stem))
if not candidates:
    raise SystemExit("No labeled image-label pairs found.")
print(f"Labeled pairs: {len(candidates)}, skipped(no image match): {skipped}")

# 3) 초기 카운트(원본)
base_cnt = current_counts(BASE)
print("Base counts:", dict(base_cnt))

# 4) 타깃/결손 세팅
cur = Counter(base_cnt)
# TARGET이 현재보다 작은 경우는 결손 0으로 간주(감소 작업은 하지 않음)
deficit = Counter({c: max(0, TARGET.get(c, 0) - cur.get(c, 0)) for c in ALL_CLASS_IDS})

use_count = defaultdict(int)
log_path = OUT_DIR / "log.csv"
with open(log_path, "w", newline="") as f:
    w = csv.writer(f)
    w.writerow(["src_img","src_lbl","out_img","out_lbl","add_per_class","totals_after"])

MAX_ITERS = 200000
iters = 0

def score(deficit: Counter, fcnt: Counter):
    # 남은 결손에 기여도가 큰 파일 우선
    return sum(deficit[c] * fcnt.get(c, 0) for c in ALL_CLASS_IDS if deficit[c] > 0)

while any(deficit[c] > 0 for c in ALL_CLASS_IDS) and iters < MAX_ITERS:
    iters += 1
    best = None
    best_score = 0

    for img, lbl, fcnt, stem in candidates:
        cap = effective_cap_for_file(fcnt, deficit)
        if use_count[img] >= cap:
            continue
        sc = score(deficit, fcnt)
        if sc > best_score:
            best_score = sc
            best = (img, lbl, fcnt, stem)

    if not best or best_score <= 0:
        print("No more useful candidates for remaining deficits. Stopping.")
        break

    img_path, lbl_path, fcnt, stem = best
    out_stem = next_unique_name(stem, used_names, tag="dup")
    out_img = OUT_DIR / f"{out_stem}{img_path.suffix.lower()}"
    out_lbl = OUT_DIR / f"{out_stem}.txt"

    # 증강
    img = load_image(img_path)
    boxes0, cls0 = read_yolo_label(lbl_path)
    boxes0 = yolo_sanitize(boxes0)

    transformed = aug(image=img, bboxes=boxes0, class_labels=cls0)
    aug_img = transformed["image"]
    aug_boxes = transformed["bboxes"]
    aug_cls = transformed["class_labels"]

    keep_boxes, keep_cls = [], []
    for (x, y, w, h), c in zip(aug_boxes, aug_cls):
        if w > 0 and h > 0:
            keep_boxes.append([float(x), float(y), float(w), float(h)])
            keep_cls.append(int(float(c)))

    # 실제로 부족 클래스에 기여하는지 확인
    test_cnt = Counter(keep_cls)
    if sum(test_cnt[c] for c in ALL_CLASS_IDS if deficit[c] > 0) <= 0 or not keep_cls:
        use_count[img_path] += 1
        continue

    cv2.imwrite(str(out_img), aug_img)
    write_yolo_label(out_lbl, keep_boxes, keep_cls, allow_ids=ALLOW_IDS)

    _, saved_cls = read_yolo_label(out_lbl)
    add_cnt = Counter(saved_cls)

    cur.update(add_cnt)
    for c in ALL_CLASS_IDS:
        deficit[c] = max(0, TARGET.get(c, 0) - cur.get(c, 0))
    use_count[img_path] += 1

    add_str = "{" + ", ".join(f"{k}:{add_cnt.get(k,0)}" for k in ALL_CLASS_IDS) + "}"
    tot_str = "{" + ", ".join(f"{k}:{cur.get(k,0)}" for k in ALL_CLASS_IDS) + "}"
    with open(log_path, "a", newline="") as f:
        w = csv.writer(f)
        w.writerow([str(img_path), str(lbl_path), str(out_img), str(out_lbl), add_str, tot_str])

    if iters % 50 == 0 or all(deficit[c]==0 for c in ALL_CLASS_IDS):
        print(f"[{iters}] cur={dict(cur)} deficit={dict(deficit)} last={out_stem}")

print("\n=== Finished ===")
print("Final counts:", dict(cur))
print("Deficit:", dict(deficit))
print(f"Saved to: {OUT_DIR}")
print(f"Log: {log_path}")

In [ ]:
# === 8클래스 YOLO 데이터 분할(Train/Val/Test) + 리포트/복사/데이터셋 YAML 생성 ==========================
from pathlib import Path
from collections import Counter, defaultdict
import random, shutil

# ====== 설정 ======
SRC = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20250909_merge_data")
DST = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20250909_merge_data_split")

# 이미지 확장자
IMG_EXTS = [".jpg", ".jpeg", ".png"]

# 분할 비율
RATIOS = {
    "train": 0.77,
    "val":   0.13,
    "test":  0.10,  # 필요 시 조정/0으로 비활성화 가능
}

# 클래스 ID (0~7)
# 0: Divot, 1: Fixed_Divot, 2: Dieased_Grass, 3: Pole,
# 4: Obstacle, 5: Sprinkler, 6: Drain, 7: Golf ball
LABELS = list(range(8))

# 배경 표기
BG = "bg"

SEED = 42
random.seed(SEED)

# ====== 유틸 ======
def ensure_dirs():
    for s, r in RATIOS.items():
        if r <= 0:
            continue
        (DST / "images" / s).mkdir(parents=True, exist_ok=True)
        (DST / "labels" / s).mkdir(parents=True, exist_ok=True)

def find_image(stem: str):
    """SRC 직하 + 재귀에서 stem과 확장자가 일치하는 '첫 이미지' 반환 (파일명 유니크 가정)"""
    for ext in IMG_EXTS:
        p = SRC / f"{stem}{ext}"
        if p.exists():
            return p
    for ext in IMG_EXTS:
        hits = list(SRC.rglob(f"{stem}{ext}"))
        if hits:
            return hits[0]
    return None

def parse_label_file(p: Path):
    """해당 파일에 등장한 '클래스 집합'을 반환. 비어있으면 빈 set."""
    labs = set()
    try:
        with open(p, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    try:
                        cid = int(float(parts[0]))
                    except:
                        continue
                    if cid in LABELS:
                        labs.add(cid)
    except:
        pass
    return labs

# ====== 1) 수집: (stem, img_path, lbl_path or None, labels_set) ======
items = []
stems = set()

# 상위/하위 폴더 전체에서 이미지 수집
for ext in IMG_EXTS:
    for img in SRC.rglob(f"*{ext}"):
        stems.add(img.stem)

for stem in sorted(stems):
    img = find_image(stem)
    if img is None:
        continue
    lbl = img.with_suffix(".txt")
    if not lbl.exists():
        # 라벨 파일이 없으면 배경으로 취급
        items.append((stem, img, None, {BG}))
    else:
        labs = parse_label_file(lbl)
        if labs:
            items.append((stem, img, lbl, labs))
        else:
            # 빈 라벨 파일이면 배경으로 취급 (YOLO는 .txt 없으면 배경 처리 → 복사 시 라벨 생략)
            items.append((stem, img, None, {BG}))

if not items:
    raise SystemExit("No images found in source folder.")

all_labels = LABELS + [BG]
N = len(items)

# ====== 2) 목표치 계산 (샘플 개수 기준) ======
target_sizes = {s: int(round(N * r)) for s, r in RATIOS.items()}

# 합 보정(반올림 오차 보정)
while sum(target_sizes.values()) < N:
    k = min(target_sizes, key=lambda s: target_sizes[s])
    target_sizes[k] += 1
while sum(target_sizes.values()) > N:
    k = max(target_sizes, key=lambda s: target_sizes[s])
    target_sizes[k] -= 1

# 전체 라벨 분포 집계 (멀티라벨: 이미지 하나가 여러 라벨을 가질 수 있음)
global_counts = Counter()
for _, _, _, labs in items:
    for l in labs:
        global_counts[l] += 1

# 분할별 라벨 목표(비율 유지)
label_targets = {s: {l: global_counts[l]*RATIOS[s] for l in all_labels} for s in RATIOS}

# ====== 3) Iterative stratification ======
# 각 라벨이 포함된 샘플 인덱스 집합
idxs_by_label = {l: set() for l in all_labels}
for i, it in enumerate(items):
    for l in it[3]:
        idxs_by_label[l].add(i)

assigned = {s: [] for s in RATIOS}
split_sizes = {s: 0 for s in RATIOS}
split_label_counts = {s: Counter() for s in RATIOS}
unassigned = set(range(N))

def cap_ok(s):
    return split_sizes[s] < target_sizes[s]

def need(s, l):
    return label_targets[s][l] - split_label_counts[s][l]

def pick_label():
    # 남은 필요량이 큰 라벨 + 희소성 가중치 고려
    best_l, best_score = None, -1e18
    for l in all_labels:
        if not (idxs_by_label[l] & unassigned):
            continue
        pos_need_sum = sum(max(0.0, need(s, l)) for s in RATIOS if cap_ok(s))
        rarity_w = 1.0 / max(1.0, global_counts[l])  # 희소할수록 가중↑
        score = pos_need_sum * rarity_w
        if score > best_score:
            best_score, best_l = score, l
    return best_l

def assign_idx_to_best_split(i):
    labs = items[i][3]
    best_s, best_cost = None, 1e18
    for s in RATIOS:
        if not cap_ok(s):
            continue
        # 라벨 오차 변화 + 사이즈 편차(가벼운 가중치)
        label_term = 0.0
        for l in labs:
            cur = split_label_counts[s][l]
            tgt = label_targets[s][l]
            label_term += abs((cur+1) - tgt) - abs(cur - tgt)
        size_term = abs((split_sizes[s]+1) - target_sizes[s]) - abs(split_sizes[s] - target_sizes[s])
        cost = label_term + 0.05*size_term
        if cost < best_cost:
            best_cost, best_s = cost, s
    if best_s is None:
        return False
    assigned[best_s].append(i)
    split_sizes[best_s] += 1
    for l in labs:
        split_label_counts[best_s][l] += 1
    unassigned.discard(i)
    for l in labs:
        idxs_by_label[l].discard(i)
    return True

# 메인 루프
while unassigned:
    l = pick_label()
    if l is None:
        break
    pool = list(idxs_by_label[l] & unassigned)
    random.shuffle(pool)

    # l 라벨이 더 필요한 split 순으로 채우기
    splits_by_need = sorted([s for s in RATIOS if cap_ok(s)], key=lambda s: need(s, l), reverse=True)
    progressed = False
    for s in splits_by_need:
        best_i, best_cost = None, 1e18
        for i in pool:
            labs = items[i][3]
            label_term = 0.0
            for ll in labs:
                cur = split_label_counts[s][ll]
                tgt = label_targets[s][ll]
                label_term += abs((cur+1) - tgt) - abs(cur - tgt)
            size_term = abs((split_sizes[s]+1) - target_sizes[s]) - abs(split_sizes[s] - target_sizes[s])
            cost = label_term + 0.05*size_term
            if cost < best_cost:
                best_cost, best_i = cost, i
        if best_i is not None:
            assigned[s].append(best_i)
            split_sizes[s] += 1
            for ll in items[best_i][3]:
                split_label_counts[s][ll] += 1
            unassigned.discard(best_i)
            for ll in items[best_i][3]:
                idxs_by_label[ll].discard(best_i)
            pool.remove(best_i)
            progressed = True
            if not cap_ok(s):
                continue
    if not progressed:
        break

# 남은 샘플은 사이즈/라벨 오차 최소 split로 배정
if unassigned:
    rest = list(unassigned)
    random.shuffle(rest)
    for i in rest:
        assign_idx_to_best_split(i)

# ====== 4) 복사 ======
ensure_dirs()

def copy_pair(idx, split):
    stem, img, lbl, labs = items[idx]
    dst_img = DST / "images" / split / img.name
    dst_lbl = DST / "labels" / split / f"{stem}.txt"
    shutil.copy2(img, dst_img)
    if lbl and lbl.exists():
        shutil.copy2(lbl, dst_lbl)
    # bg는 라벨 파일 생성 안 함 (YOLO가 없는 경우를 배경으로 처리)

for s, idxs in assigned.items():
    for i in idxs:
        copy_pair(i, s)

# ====== 5) data.yaml (test 포함) ======
yaml_path = DST / "data.yaml"
with open(yaml_path, "w") as f:
    f.write("train: images/train\n")
    f.write("val: images/val\n")
    if target_sizes.get("test", 0) > 0:
        f.write("test: images/test\n")
    f.write("\n")
    f.write("nc: 8\n")
    f.write('names: ["Divot","Fixed_Divot","Dieased_Grass","Pole","Obstacle","Sprinkler","Drain","Golf ball"]\n')

# ====== 6) 리포트 (train/val/test 모두) ======
def summarize_all():
    splits = list(RATIOS.keys())
    rep = []
    total = sum(len(assigned.get(s, [])) for s in splits)
    rep.append(f"Total images: {total}")
    rep.append(f"Targets: {target_sizes}")
    rep.append("Global label counts: " + ", ".join([f"{l}:{global_counts[l]}" for l in all_labels]))
    rep.append("Global label ratios: " + ", ".join([f"{l}:{global_counts[l]/max(1,total):.3f}" for l in all_labels]))
    for s in splits:
        if s not in assigned:
            continue
        n = len(assigned[s])
        cnt = split_label_counts.get(s, Counter())
        rep.append(f"\n[{s}] images: {n} (target {target_sizes.get(s, 0)})")
        rep.append("  label counts: " + ", ".join([f"{l}:{cnt[l]}" for l in all_labels]))
        rep.append("  label ratios: " + ", ".join([f"{l}:{cnt[l]/max(1,n):.3f}" for l in all_labels]))
        # 타깃 대비 라벨 분포 오차(참고용)
        errs = []
        for l in all_labels:
            tgt = label_targets[s][l]
            err = cnt[l] - tgt
            rel = 0.0 if tgt == 0 else err / max(1.0, tgt)
            errs.append(f"{l}:Δ{err:.1f}({rel:+.1%})")
        rep.append("  errors vs target: " + ", ".join(errs))
    return "\n".join(rep)

report = summarize_all()
print(report)
(DST / "split_report.txt").write_text(report, encoding="utf-8")

print(f"\n✅ Done. Split at: {DST}")
print(f"↳ Saved: {yaml_path}, split_report.txt")


In [1]:
from ultralytics import YOLO

# 데이터 설정: train/val 경로, 클래스 수/이름이 정확해야 함
DATA_YAML = "/home/dw/ws_job_msislab/Golf_Project/data/for_study/20250909_merge_data_split/data.yaml"

# 모델 선택: s(가볍고 빠름) → 결과 좋아지면 m으로 확장
model = YOLO("yolo11s.pt")  # ← 여기만 바뀌면 됩니다

results = model.train(
    # === 기본 학습 설정 ===
    data=DATA_YAML,      # 학습/검증 경로 정의된 data.yaml
    epochs=500,          # 총 학습 epoch (조기종료가 있으면 실제로는 더 적게 끝날 수 있음)
    patience=50,         # val metric이 개선 없을 때 조기종료 대기 epoch
    imgsz=1024,          # 입력 해상도(정사각). 소물체 ↑. OOM 시 896/768로
    batch=-1,            # 자동 배치 크기 탐색. OOM 나면 8~16으로 수동 고정
    device=0,            # GPU 인덱스 (여러 GPU면 "0,1" 형태 가능)
    workers=8,           # DataLoader 병렬 로드 워커 수 (디스크/CPU 상황에 맞춰)
    rect=True,           # aspect ratio 유지한 사각형 배치 → 소물체 왜곡↓, 수렴 안정화
    cache=True,          # 이미지/라벨 메모리 캐시. RAM 부족 시 False 권장
    amp=True,            # 자동 혼합정밀도(fp16). 최신 GPU에서 속도/메모리 이점

    # === 온라인 증강(가볍게) ===
    mosaic=0.10,         # 초반 모자이크 비율. 소물체 리콜↑. 과하면 artifacts↑
    close_mosaic=15,     # 지정 epoch 이후 mosaic=0 자동 전환(과적합 방지)
    mixup=0.0,           # MixUp 비활성(디봇/잔디 경계 왜곡 방지 목적)
    copy_paste=0.0,      # Copy-Paste 비활성(객체 경계/텍스처 왜곡 우려)
    hsv_h=0.01,          # Hue 변동(작게). 잔디 색 왜곡 최소화
    hsv_s=0.2,           # Saturation 변동
    hsv_v=0.2,           # Value(밝기) 변동
    degrees=0.0,         # 회전 금지(지형/그림자 방향성 유지 목적)
    translate=0.0,       # 이동 변형 금지(필요 시 0.05~0.1 권장)
    scale=0.0,           # 스케일 변형 금지(필요 시 0.10~0.20 권장)
    shear=0.0,           # 기울이기 금지(텍스처 왜곡 방지)
    erasing=0.0,         # 랜덤 Erasing 비활성(작은 객체 소실 방지)
    # ※ 오프라인 증강을 이미 다수 수행했다면 온라인은 약하게 유지하는 편이 안전

    # === 손실/최적화 쪽 튜닝 ===
    box=10.0,            # bbox 회귀 손실 가중(기본≈7.5). 위치 정밀도↑
    cls=0.5,             # 클래스 분류 손실 가중(기본 수준)
    dfl=1.5,             # Distribution Focal Loss 가중(경계 정밀도↑)
    # lr0/lrf: Cosine 스케줄(초기 lr → 최종 lr=lr0*lrf)
    lr0=0.002,           # 초기 lr (AdamW 기준 안전값 / SGD면 보통 0.01)
    lrf=0.01,            # 최종 lr 비율 (0.01이면 마지막에 lr0의 1%)
    momentum=0.9,        # SGD: 모멘텀, Adam/AdamW: beta1 역할로 사용됨(ultra 내부 호환)
    weight_decay=0.01,   # AdamW 권장: 0.01 / SGD 권장: 0.0005

    # === Optimizer 선택 ===
    # optimizer 인자를 생략하면 기본 SGD로 동작
    # optimizer="SGD",     # 대규모 데이터/보수적 일반화. 권장 세팅: lr0=0.01, wd=0.0005, momentum=0.937
    optimizer="AdamW",    # 소/중규모+증강 多에서 빠르고 안정적 수렴. 권장: lr0≈0.002, wd≈0.01
    # optimizer="Adam",    # 빠른 수렴(초반 성능↑)이나 과적합 주의. 권장: lr0≈0.001~0.002, wd 0~0.0005
    # optimizer="RMSProp", # 특수 케이스 외 권장X. 튜닝 민감.
    # optimizer="NAdam",   # Adam+Nesterov. 드물게 사용. 실험용.

    # === 출력 경로 ===
    project="/home/dw/ws_job_msislab/Golf_Project/runs_yolo",  # 상위 폴더
    name="202509010_merge_data_yolov11s",  # 실험 이름(여러 실험 구분용)
    exist_ok=True            # 같은 이름 덮어쓰기 허용
)

     26/500      2.62G      1.912      1.228      1.483          7       1024: 100%|██████████| 737/737 [01:06<00:00, 11.12it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 63/63 [00:03<00:00, 16.15it/s]

                   all        498       1562      0.582      0.595      0.616      0.285



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     27/500      2.62G      1.859      1.172      1.459          7       1024: 100%|██████████| 737/737 [01:05<00:00, 11.31it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 63/63 [00:03<00:00, 16.65it/s]

                   all        498       1562      0.656      0.569      0.602      0.277


...

    381/500      2.62G      0.287     0.1645     0.7438          7       1024: 100%|██████████| 737/737 [01:04<00:00, 11.39it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 63/63 [00:03<00:00, 16.43it/s]

                   all        498       1562      0.889       0.72      0.853      0.677
EarlyStopping: Training stopped early as no improvement observed in last 50 epochs. Best results observed at epoch 331, best model saved as best.pt.
To update EarlyStopping(patience=50) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



381 epochs completed in 7.309 hours.
Optimizer stripped from /home/dw/ws_job_msislab/Golf_Project/runs_yolo/202509010_merge_data_yolov11s/weights/last.pt, 19.3MB
Optimizer stripped from /home/dw/ws_job_msislab/Golf_Project/runs_yolo/202509010_merge_data_yolov11s/weights/best.pt, 19.3MB

Validating /home/dw/ws_job_msislab/Golf_Project/runs_yolo/202509010_merge_data_yolov11s/weights/best.pt...
Ultralytics 8.3.175 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7932MiB)
YOLO11s summary (fused): 100 layers, 9,415,896 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 63/63 [00:03<00:00, 18.41it/s]


                   all        498       1562      0.857      0.769      0.872      0.682
                 Divot        357        790      0.876      0.814      0.885      0.722
           Fixed_Divot        268        609      0.865      0.819      0.896      0.697
         Dieased_Grass         48         76      0.902      0.263       0.61      0.481
                  Pole         28         28      0.745       0.94      0.929      0.618
             Sprinkler          6          6      0.742          1      0.955      0.856
                 Drain         25         25      0.912      0.829      0.902      0.746
             Golf ball         28         28       0.96      0.714      0.928      0.655
Speed: 0.4ms preprocess, 5.2ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /home/dw/ws_job_msislab/Golf_Project/runs_yolo/202509010_merge_data_yolov11s
